In [1]:
pip install keras-tuner


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python-headless


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scikit-learn


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install imgaug


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import pandas as pd
import cv2
import imgaug.augmenters as iaa
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, concatenate
from sklearn.model_selection import train_test_split
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.utils import Sequence
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

class AugmentedDataGenerator(Sequence):
    def __init__(self, X, y, batch_size=32, augmentations=None):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.augmentations = augmentations

    def __len__(self):
        return int(np.ceil(len(self.X) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.X[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        if self.augmentations:
            batch_x = augment_images(batch_x)

        return batch_x, batch_y

# Define the augmentations using imgaug
augmentation = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontally flip 50% of the images
    iaa.Rotate((-10, 10)), # rotate images between -10 and 10 degrees
])

# Apply augmentations to the images
def augment_images(images):
    return np.array([augmentation.augment_image(image) for image in images])

# load train dataset
train_df = pd.read_csv('data/training_norm.csv')
train_images = []
for image_id in train_df['image_id']:
    image = cv2.imread(f'data/training_data/training_data/{image_id}.png')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    train_images.append(image)
train_images = np.array(train_images)
train_angles = np.array(train_df['angle'])
train_speeds = np.array(train_df['speed'])

# Apply augmentations to the train images
train_images = augment_images(train_images)

# load test dataset
test_images = []
for i in range(1, 1021):
    image = cv2.imread(f'data/test_data/test_data/{i}.png')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    test_images.append(image)
test_images = np.array(test_images)
test_ids = np.arange(1, 1021)


# split the data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images, train_df[['angle', 'speed']].values, test_size=0.2, random_state=42)

# Apply augmentations to the train set
# X_train = augment_images(X_train)

# define the early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)


2023-03-31 00:52:01.883341: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 00:52:02.977814: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /environment/miniconda3/lib/python3.8/site-packages/cv2/../../lib64:
2023-03-31 00:52:02.977908: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /environment/miniconda3/lib/python3.8/site-packages/cv2/../..

In [6]:

# define the model
def build_model(hp):
    # input layer
    input_layer = Input(shape=(240, 320, 3))

    # convolutional layers
    num_conv_layers = hp.Int('num_conv_layers', min_value=2, max_value=5, step=1)
    conv_layer_filters = [hp.Choice(f'conv_layer_{i}_filters', values=[32, 64, 128, 256], ordered=False, default=64) for i in range(num_conv_layers)]
    conv_layer_kernel_sizes = [hp.Choice(f'conv_layer_{i}_kernel_sizes', values=[3, 5, 7], ordered=False, default=3) for i in range(num_conv_layers)]
    conv_layer_activations = [hp.Choice(f'conv_layer_{i}_activations', values=['relu', 'elu'], default='relu') for i in range(num_conv_layers)]
    x = input_layer
    for i in range(num_conv_layers):
        x = Conv2D(filters=conv_layer_filters[i], kernel_size=conv_layer_kernel_sizes[i], activation=conv_layer_activations[i], padding='same')(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)

    # fully connected layers
    x = Flatten()(x)
    num_dense_layers = hp.Int('num_dense_layers', min_value=1, max_value=3, step=1)
    for i in range(num_dense_layers):
        dense_layer_units = hp.Int(f'dense_layer_{i}_units', min_value=32, max_value=512, step=32)
        dense_layer_activation = hp.Choice(f'dense_layer_{i}_activation', values=['relu', 'elu'], default='relu')
        x = Dense(units=dense_layer_units, activation=dense_layer_activation)(x)
        dropout_rate = hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1, default=0.25)
        x = Dropout(rate=dropout_rate)(x)

    # output layer
    angle_output = Dense(units=1, name='angle_output')(x)
    speed_output = Dense(units=1, name='speed_output')(x)
    model = Model(inputs=input_layer, outputs=[angle_output, speed_output])

    # compile the model
    optimizer = Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log'))
    loss = {'angle_output': 'mean_squared_error', 'speed_output': 'mean_squared_error'}
    metrics = {'angle_output': 'mae', 'speed_output': 'mae'}
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

# define the tuner
tuner = RandomSearch(
build_model,
objective='val_loss',
max_trials=10,
directory='output',
project_name='steering_angle_regression'
)


# search for the best hyperparameters
tuner.search(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[early_stop])

# get the best model
best_model = tuner.get_best_models(num_models=1)[0]
# save the best model as an .h5 file
best_model.save('best_model.h5')





INFO:tensorflow:Reloading Tuner from output/steering_angle_regression/tuner0.json
INFO:tensorflow:Oracle triggered exit


2023-03-31 00:52:45.369011: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-31 00:52:45.441964: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-31 00:52:45.442150: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-31 00:52:45.443586: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [7]:
# Create the data generator for the training set
train_generator = AugmentedDataGenerator(X_train, y_train, batch_size=32, augmentations=augmentation)

# Train the model using the generator
history = best_model.fit(train_generator, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stop])
y_val_pred = best_model.predict(X_val)
angle_r2 = r2_score(y_val[:, 0], y_val_pred[0].flatten())
angle_rmse = np.sqrt(mean_squared_error(y_val[:, 0], y_val_pred[0].flatten()))
angle_mape = mean_absolute_percentage_error(y_val[:, 0], y_val_pred[0].flatten())
speed_r2 = r2_score(y_val[:, 1], y_val_pred[1].flatten())
speed_rmse = np.sqrt(mean_squared_error(y_val[:, 1], y_val_pred[1].flatten()))
speed_mape = mean_absolute_percentage_error(y_val[:, 1], y_val_pred[1].flatten())

print("Angle R-squared:", angle_r2)
print("Angle RMSE:", angle_rmse)
print("Angle MAPE:", angle_mape)
print("Speed R-squared:", speed_r2)
print("Speed RMSE:", speed_rmse)
print("Speed MAPE:", speed_mape)
# train the model on the entire training dataset
history = best_model.fit(train_images, [train_angles, train_speeds], epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100


2023-03-31 00:52:49.814402: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8101
2023-03-31 00:52:51.659506: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-31 00:52:51.675314: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2023-03-31 00:52:51.675343: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:237] Used ptxas at /usr/local/cuda-11.2/bin/ptxas
2023-03-31 00:52:51.675401: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:318] UNIMPLEMENTED: /usr/local/cuda-11.2/bin/ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-03-31 00:52:53.185097: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] Te

345/345 [==============================] - 24s 44ms/step - loss: 0.1436 - angle_output_loss: 0.0716 - speed_output_loss: 0.0720 - angle_output_mae: 0.2332 - speed_output_mae: 0.2335 - val_loss: 0.1392 - val_angle_output_loss: 0.0696 - val_speed_output_loss: 0.0696 - val_angle_output_mae: 0.2318 - val_speed_output_mae: 0.2318
Epoch 2/100
345/345 [==============================] - 14s 40ms/step - loss: 0.1415 - angle_output_loss: 0.0706 - speed_output_loss: 0.0709 - angle_output_mae: 0.2320 - speed_output_mae: 0.2320 - val_loss: 0.1362 - val_angle_output_loss: 0.0680 - val_speed_output_loss: 0.0682 - val_angle_output_mae: 0.2307 - val_speed_output_mae: 0.2307
Epoch 3/100
345/345 [==============================] - 14s 41ms/step - loss: 0.1389 - angle_output_loss: 0.0693 - speed_output_loss: 0.0695 - angle_output_mae: 0.2311 - speed_output_mae: 0.2311 - val_loss: 0.1365 - val_angle_output_loss: 0.0682 - val_speed_output_loss: 0.0683 - val_angle_output_mae: 0.2311 - val_speed_output_mae: 0.

In [8]:

# make predictions on the test set
test_predictions = best_model.predict(test_images)

# get the steering angle and speed predictions separately
test_steering_predictions = test_predictions[0]
test_speed_predictions = test_predictions[1]

# speed adjustment
if np.min(test_speed_predictions) < 0:
    test_speed_predictions = test_speed_predictions - np.min(test_speed_predictions) #if there is negative num in spd column
    max_val = np.max(test_speed_predictions)      
    normalized_arr_spd = test_speed_predictions / max_val
    test_speed_predictions = np.where(normalized_arr_spd >= 0.5, 1, 0)
else:
    max_val = np.max(test_speed_predictions)      
    normalized_arr_spd = test_speed_predictions / max_val
    test_speed_predictions = np.where(normalized_arr_spd >= 0.5, 1, 0)

submission_df = pd.DataFrame({
    'image_id': test_ids.flatten(),
    'angle': test_steering_predictions.flatten(),
    'speed': test_speed_predictions.flatten()
})
submission_df.to_csv('output/submission.csv', index=False)


32/32 [==============================] - 1s 19ms/step


In [9]:
# speed adjustment
if np.min(test_speed_predictions) < 0:
    test_speed_predictions = test_speed_predictions - np.min(test_speed_predictions) #if there is negative num in spd column
    max_val = np.max(test_speed_predictions)      
    normalized_arr_spd = test_speed_predictions / max_val
    test_speed_predictions = np.where(normalized_arr_spd >= 0.5, 1, 0)
else:
    max_val = np.max(test_speed_predictions)      
    normalized_arr_spd = test_speed_predictions / max_val
    test_speed_predictions = np.where(normalized_arr_spd >= 0.5, 1, 0)